In [25]:
"""

MJ Price Scraper Tool v1.0

"""

# List of dispensaries to scrape and preset filters to pick from

dispos = {
    "Berkshire Roots" : "https://dutchie.com/dispensary/berkshire-roots-east-boston",
    "Mission Brookline" : "https://dutchie.com/dispensary/mission-brookline",
    "NETA" : "https://dutchie.com/dispensary/neta-brookline",
    "Western Front" : "https://dutchie.com/dispensary/western-front1",
    "MedMen" : "https://dutchie.com/dispensary/medmen-fenway",
    "Rise" : "https://dutchie.com/dispensary/greenstar-herbals-inc-chelsea",
    "Happy Valley" : "https://dutchie.com/dispensary/happy-valley-east-boston",
    "Pure Oasis" : "https://dutchie.com/dispensary/pure-oasis1",
    "Ayr" : "https://dutchie.com/dispensary/ayr-dispensary-back-bay"
}

filters = {
    "Concentrate" : "/products/concentrates?sortby=pricelowtohigh",
    "Flower" : "/products/flower?potencycbd=0%2C50&potencythc=0%2C50&sortby=pricelowtohigh",
    "Carts" : "/products/vaporizers?sortby=pricelowtohigh"
}

# Change this variable to pick the product you are looking for

filter = filters["Carts"]


In [26]:
# [1]
# Imports
import requests
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Session options
options = Options()
options.page_load_strategy = 'normal'
options = webdriver.ChromeOptions()
#driver = webdriver.Chrome('chromedriver',options=chrome_options)
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [27]:
#[2]
# Product class w/ attrs.
class Product:
  def __init__(self, name, company, type, thc, weight, price, oldprice, discount, dispo):
    self.name = name
    self.company = company
    self.type = type
    self.thc = thc
    self.weight = weight
    self.price = price
    self.oldprice = oldprice
    self.discount = discount
    self.dispo = dispo
    

In [28]:
#[3]
# Function to scrape URL and returns the main element containing products and prod info
def scrapeURL(driver, dispo, filter): #DONE: Add filter var

    # Connect to URL
    filterURL = dispos[dispo] + filter

    # temp fix to other dispos using 3.5g vs 1/8oz as weight filter (No Longer Needed)
    # if (dispo == "MedMen" & filter.contains("Flower1_8")):
    #    filterURL = dispos[dispo] + filters["Flower3_5, Ascending Price"]


    driver.get(filterURL)
    driver.implicitly_wait(1.5)
    #time.sleep(5)

    # Age verification
    
    try:
        driver.find_element(By.CLASS_NAME, "age-confirmation-modal__StyledButton-di8wrk-0").click()
    except:
        print("")


    # Smooth scroll to scrub ALL data
    scheight = .1
    while scheight < 9.9:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight/%s);" % scheight)
        scheight += .0075

    # Main element with all products
    main_element = driver.find_element(By.TAG_NAME, "main")
    
    # Searching for the product lists
    element_products = main_element.find_element(By.CLASS_NAME, "product-list__Container-sc-1arkwfu-1")
    element_div = element_products.find_element(By.TAG_NAME, "div")
    element_prod_list = element_div.find_elements(By.CLASS_NAME, "mobile-product-list-item__Container-zxgt1n-0")
    prod_list = element_prod_list[1:len(element_prod_list)-1]

    # Parsing the list into organized Product class
    products = []
    for element in prod_list:
        data = element.text.split("\n")
        if (len(data) == 7):   # Discounted Product
            p = Product(data[0], data[1], data[2], data[2], data[3], data[5], data[4], data[6], dispo)
            products.append(p)
        elif (len(data) == 6): # Staff Pick Product
            p = Product(data[1], data[2], data[3], data[3], data[4], data[5], data[5], "0%", dispo)
            products.append(p)
        elif (len(data) == 5): # Regular Product
            p = Product(data[0], data[1], data[2], data[2], data[3], data[4], data[4], "0%", dispo)
            products.append(p)
    
    print(len(products), "products scraped from", dispo)
    return products

In [29]:
#[4]
products = []
i=0
for dispo in dispos: #TODO: Fix Happy Valley prices
    dispo_products = scrapeURL(driver, dispo, filter)
    products.extend(dispo_products)
    i+=1
    print("[", i, "/", len(dispos), "] Dispos scraped...")

11 products scraped from Berkshire Roots
[ 1 / 9 ] Dispos scraped...

33 products scraped from Mission Brookline
[ 2 / 9 ] Dispos scraped...

34 products scraped from NETA
[ 3 / 9 ] Dispos scraped...

1 products scraped from Western Front
[ 4 / 9 ] Dispos scraped...

22 products scraped from MedMen
[ 5 / 9 ] Dispos scraped...

56 products scraped from Rise
[ 6 / 9 ] Dispos scraped...

28 products scraped from Happy Valley
[ 7 / 9 ] Dispos scraped...

35 products scraped from Pure Oasis
[ 8 / 9 ] Dispos scraped...

54 products scraped from Ayr
[ 9 / 9 ] Dispos scraped...


In [30]:
#[5]
p = products
p.sort(key=lambda x: x.price, reverse=False)

print(len(p), "products:")

for pr in p:
        print(pr.name, pr.price, pr.weight, pr.dispo)

#TODO: Add deals from dispo home blurb

#TODO: Find best deals and other info

#TODO: Clean names


#TODO: add 1g version of .5g products

#TODO: Add custom function for Sanctuary

#TODO: Categorize (Wax, Sugar, Kief, Crumble, ...)

#TODO: Data visualization (tables and graphs)

#TODO: Store logs of data w/ timestamp


274 products:
Blue Mirage Vape Cartridge | 1g $100.00 1g - MedMen
GG#4 Vape Cartridge | 1g $100.00 1g - MedMen
Berry Haze Cartridge $100.00 1g - Ayr
Muddled Mojo Cartridge | .5g - Basics $19.95 0.5 - Berkshire Roots
Sour D Cartridge | .5g - Basics $19.95 0.5 - Berkshire Roots
Spunzleberry Sugar Cartridge | .5g - Basics $19.95 0.5 - Berkshire Roots
Tangie CBD 2:1 Cartridge | .5g $19.95 0.5 - Berkshire Roots
Tangie Cartridge | .5g - Basics $19.95 0.5 - Berkshire Roots
Zkittlez Cartridge | .5g - Basics $19.95 0.5 - Berkshire Roots
Black Lime (Hybrid-Indica) - Disposable Vaporizer Pen, 0.3g $21.00 .3g - NETA
Jack Herer (Hybrid-Sativa) - Disposable Vaporizer Pen, 0.3g $21.00 .3g - NETA
Maui Wowie (Sativa) - Disposable Vaporizer Pen, 0.3g $21.00 .3g - NETA
Mazar x Blueberry (Hybrid) - Disposable Vaporizer Pen, 0.3g $21.00 .3g - NETA
Purple Punch (Indica) - Disposable Vaporizer Pen, 0.3g $21.00 .3g - NETA
Caribbean Sunshine (Indica) - Disposable Vaporizer Pen, 0.3g $24.00 .3g - NETA
Far Side 